<a href="https://colab.research.google.com/github/colinZejda/Summer2023_UCI_ML_Research/blob/main/2_CNN_from_scratch_my_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import torch
import torch.nn as nn
import torchvision
from tqdm.autonotebook import tqdm

<ipython-input-1-330b78dc0769>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# transform code (makes image digestible for model)
# normalizing data makes training faster, easier to converge
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# extra transorm for training data, hopefully achieves better performance
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    torchvision.transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    torchvision.transforms.RandomHorizontalFlip()
])

In [ ]:
# get the datasets
# these are iterable objects of paths (iterable only if we enhance it a bit)
    # this enhancement is the data loader below (or we could use a list of indices)
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform)
validation_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=train_transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# data loaders function, loads training, validation and test data
# how is this dif from the block above?
    # it will return the data in a specific batch size, maybe shuffle it a bit
    # data loaders-- allow us to iterate through the data in batches (no for loop needed)
    # if the data is huge, we can load a bit at a time
def data_loaders(train_set, validation_set, test_set, size):
  trainloader = torch.utils.data.DataLoader(train_set, batch_size=size, num_workers=1)
  validloader = torch.utils.data.DataLoader(validation_set, batch_size=size*2, num_workers=1)
  testloader = torch.utils.data.DataLoader(test_set, batch_size=size, num_workers=1)
  return trainloader, validloader, testloader

In [ ]:
# defining the model
class my_model(nn.Module):
  def __init__(self):
    super(my_model, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)          # 3 input channels, 64 output channels
    self.leakyrelu = nn.LeakyReLU()
    self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
    self.fc1 = nn.Linear(8192, 1000)
    self.fc2 = nn.Linear(1000, 10)                                               # output is 10 bc there are 10 classes to classify

  def forward(self, x):
    x = self.conv1(x)
    x = self.leakyrelu(x)
    x = self.conv2(x)
    x = self.leakyrelu(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    x = self.fc2(x)
    return x


In [ ]:
# set hyperparameters
num_classes = 10
num_epochs = 10
batch_size = 16
learning_rate = 1e-3

# load dataset batches that we want + instantiate model
train, validation, test = data_loaders(train_set, validation_set, test_set, batch_size)
model = my_model()

# loss and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# training
for epoch_i in range(num_epochs):
  for images, labels in tqdm(train):
    if images.shape[0] == batch_size:
      # forward pass + compute loss
      y_hat = model.forward(images)
      loss = loss_func(y_hat, labels)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  print('Epoch {}/{}, Loss: {:.3f}'.format(epoch_i+1, num_epochs, loss.item()))

  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 1/10, Loss: 2.245


  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 2/10, Loss: 2.225


  0%|          | 0/3125 [00:00<?, ?it/s]

...

Epoch 8/10, Loss: 1.961


  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 9/10, Loss: 1.781


  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 10/10, Loss: 1.802


In [ ]:
# VALIDATION
with torch.no_grad():
  acc_count = 0
  total = 0
  for images, labels in test:
    # print(images.shape)
    if images.shape[0] == batch_size:      # at the end we may not have enough images for a batch, just ignore them
      y_hat = model(images)
      _, predicted = torch.max(y_hat.data, 1)
      total += labels.size(0)
      acc_count += (predicted == labels).sum().item()
  print('Accuracy on the test images: {} %'.format(100 * acc_count / total))

Accuracy on the test images: 43.71 %
